#Práctica 3
###David Davó Laviña & Ela Katherine Shepherd Arévalo

In [ ]:
!pip install -q qwikidata

Vamos a definir cosas magicas de Jupyter para hacernos más fácil las tareas, e incluir el sparql como código (aunque sin coloreado sintáctico)

In [ ]:
import json
import pandas as pd
from qwikidata.sparql import return_sparql_query_results
from IPython.core.magic import register_cell_magic

@register_cell_magic
def wdsparql(line, cell):
    """ Ejecuta directamente consultas SPARQL en wikidata """
    result = return_sparql_query_results(cell)

    if line == "debug": print(json.dumps(result, indent=2))
    
    out = [[None if r.get(c) is None else r.get(c)["value"] for c in result['head']['vars']] for r in result['results']['bindings']]

    return pd.DataFrame(out, columns=result['head']['vars'])

In [ ]:
%%wdsparql
SELECT ?childLabel
WHERE
{
# ?child  father   Bach
  ?child wdt:P22 wd:Q1339.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 10

,childLabel
0,Johann Christoph Friedrich Bach
1,Carl Philipp Emanuel Bach
2,Johann Christian Bach
3,Wilhelm Friedemann Bach
4,Johann Gottfried Bernhard Bach
5,Gottfried Heinrich Bach
6,Catharina Dorothea Berci
7,Elisabeth Juliana Friderica Bach
8,Maria Sophia Bach
9,Léopold Augustus Bach


##CONSULTA 1

[Hans Zimmer (Q76364)](https://www.wikidata.org/wiki/Q76364) es un famoso compositor de muchas bandas sonoras de
películas. Vamos a comenzar por averiguar su fecha y lugar de nacimiento.

In [ ]:
%%wdsparql
SELECT ?birthday ?birthPlaceLabel ?birthPlaceDescription
WHERE
{
    # Hans Zimmer   birthday
    wd:Q76364 wdt:P569 ?birthday;
    #               birthplace
              wdt:P19 ?birthPlace.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
 }

,birthday,birthPlaceLabel,birthPlaceDescription
0,1957-09-12T00:00:00Z,Frankfurt am Main,"city in Hesse, Germany"


##CONSULTA 2

A continuación, vamos a averiguar los instrumentos que toca. Queremos obtener los
resultados ordenados alfabéticamente por el nombre del instrumento.

In [ ]:
%%wdsparql
SELECT ?instrumentLabel
WHERE
{
    # Hans Zimmer   instrument
    wd:Q76364 wdt:P1303 ?instrument
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }    
}
ORDER BY ASC(?instrumentLabel)

,instrumentLabel
0,banjo
1,bass guitar
2,guitar
3,keyboard instrument
4,piano
5,synthesizer


##CONSULTA 3
¿Cuales de los instrumentos que toca son instrumentos de cuerda? Ten en cuenta que los isntrumentos pueden ser indirectamente de cuerda a travéés de varias relaciones de subclase

In [ ]:
%%wdsparql
SELECT ?instrumentLabel
WHERE
{
    # Hans Zimmer   instrument
    wd:Q76364 wdt:P1303 ?instrument.
    #           any number of subclasses / string instrument
    ?instrument wdt:P279* wd:Q1798603.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }    
}
ORDER BY ASC(?instrumentLabel)

,instrumentLabel
0,banjo
1,bass guitar
2,guitar
3,piano


##CONSULTA 4
Zimmer ha recibido muchas nominaciones y premios a lo largo de su carrera. Queremos obtener una lista de nominaciones y para cada una de ellas el trabajo por el cual fue nominado. Para resolver esta consulta necesitaráás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa wikidata.

In [ ]:
%%wdsparql
SELECT ?filmLabel ?nominationLabel
WHERE
{
  # Hans Zimmer / nominated for / for work / nominated for
   wd:Q76364 p:P1411 [pq:P1686 ?film; ps:P1411 ?nomination].
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
 }
ORDER BY ?filmLabel


,filmLabel,nominationLabel
0,As Good as It Gets,Academy Award for Best Original Musical or Com...
1,Gladiator,Academy Award for Best Original Score
2,Inception,Academy Award for Best Original Score
3,Interstellar,Academy Award for Best Original Score
4,Rain Man,Academy Award for Best Original Score
5,Sherlock Holmes,Academy Award for Best Original Score
6,The Lion King,Academy Award for Best Original Score
7,The Preacher's Wife,Academy Award for Best Original Musical or Com...
8,The Prince of Egypt,Academy Award for Best Original Musical or Com...
9,The Thin Red Line,Academy Award for Best Original Dramatic Score


##CONSULTA 5
Ahora queremos conocer el título de todas las películas en las que Zimmer ha
colaborado como compositor. Se mostrarán ordenadas alfabéticamente y debes tener
cuidado de no mostrar resultados repetidos.


In [ ]:
%%wdsparql
# Nota: Algunas películas tienen varios títulos, así que agrupamos por pelicula
# y elegimos uno de ellos
SELECT (SAMPLE(?title) AS ?title)
WHERE
{   
    # ?film  instance of   movie
    ?film    wdt:P31       wd:Q11424;
    #        composer      Hans Zimmer
             wdt:P86       wd:Q76364;
    #        title         ?title
             wdt:P1476     ?title.
}
GROUP BY ?film
ORDER BY ASC(?title)

,title
0,12 Years a Slave
1,A League of Their Own
2,A World Apart
3,An Everlasting Piece
4,Angels & Demons
...,...
106,Widows
107,Winter's Tale
108,Wonder Woman 1984
109,Younger and Younger


##CONSULTA 6
Zimmer es sin duda un compositor prolífico. ¿Exactamente en cuentas películas ha
participado como compositor?

In [ ]:
%%wdsparql
SELECT (COUNT (*) as ?totalcnt)
WHERE
{   
    # ?film  instance of   movie
    ?film    wdt:P31       wd:Q11424;
    #        composer      Hans Zimmer
             wdt:P86       wd:Q76364.
}

,totalcnt
0,111


##CONSULTA 7
Es importante que las películas tengan una duración adecuada, ni muy cortas ni
demasiado largas. De todas las películas en las que ha colaborado Zimmer como
compositor, ¿cuáles duran entre 90 y 120 minutos? Muestra los resultados ordenados
alfabéticamente. Para cada película muestra el título y la duración.
Algunas películas (como Salto al peligro) pueden tener varias duraciones (supongo
que son distintas versiones de la película). En este apartado no importa si aparecen
varias veces en los resultados.

In [ ]:
%%wdsparql
SELECT (SAMPLE(?title) AS ?title) (GROUP_CONCAT(?duration;separator="; ") as ?durations)
WHERE
{   
    # ?film  instance of   movie
    ?film    wdt:P31       wd:Q11424;
    #        composer      Hans Zimmer
             wdt:P86       wd:Q76364;
    #        title         ?title
             wdt:P1476     ?title;
    #        duration      ?duration
             wdt:P2047     ?duration.
    FILTER(90 <= ?duration && ?duration <= 120)
}
GROUP BY ?film
ORDER BY ASC(?title)

,title,durations
0,A World Apart,112
1,An Everlasting Piece,108
2,Beyond Rangoon,95
3,Bird on a Wire,110
4,Broken Arrow,110
5,Chappie,120
6,Chicago Joe and the Showgirl,103
7,Chill Factor,101
8,Crimson Tide,111
9,Dark Phoenix,114


##CONSULTA 8
Vamos a recuperar ahora las películas más actuales en las que Zimmer ha colaborado
como compositor. Estamos interesados específicamente en películas que se hayan
estrenado a partir de 2005.
Seguramente, en tu primer intento verás que cada película aparece repetida varias
veces con fechas distintas porque Wikidata contiene las fechas de estreno en cada
país. Vamos a considerar que la fecha real de estreno de la película es la fecha más
antigua de todas ellas.
Para que cada película aparezca sólo una vez con la fecha correcta necesitarás agrupar
las respuestas por película y título, y aplicar una función de agregación sobre las
fechas de publicación. Los resultados se tienen que mostrar ordenados
alfabéticamente.

In [ ]:
%%wdsparql
SELECT (SAMPLE(?title) AS ?title) (MIN(?pdate) AS ?earliestpdate)
WHERE
{   
    # ?film  instance of   movie
    ?film    wdt:P31       wd:Q11424;
    #        composer      Hans Zimmer
             wdt:P86       wd:Q76364;
    #        title         ?title
             wdt:P1476     ?title;
    #        publication   ?pdate
             wdt:P577      ?pdate
}
GROUP BY ?film
# Tambien podemos usar ?earlistpdate >= "2005-01-01"^^xsd:dateTime
HAVING (YEAR(?earliestpdate) >= 2005)
ORDER BY ASC(?title)

,title,earliestpdate
0,12 Years a Slave,2013-08-30T00:00:00Z
1,Angels & Demons,2009-01-01T00:00:00Z
2,Batkid,2015-01-01T00:00:00Z
3,Batman Begins,2005-06-15T00:00:00Z
4,Batman v Superman: Dawn of Justice,2016-03-23T00:00:00Z
5,Believer,2018-01-01T00:00:00Z
6,Blade Runner 2049,2017-10-03T00:00:00Z
7,Chappie,2015-03-05T00:00:00Z
8,Dark Phoenix,2019-06-05T00:00:00Z
9,Dune,2020-01-01T00:00:00Z


##CONSULTA 9
En una película pueden colaborar varios músicos. ¿Qué otros compositores han
compuesto junto a Zimmer en una película? Para cada uno de ellos muestra su
nombre, la fecha de nacimiento y (si ha fallecido) la fecha de defunción. Los resultados
deben aparecer ordenados alfabéticamente.


In [ ]:
%%wdsparql
SELECT ?composerLabel (SAMPLE(?dob) AS ?dob) (SAMPLE(?dod) AS ?dod)
WHERE
{   
    # ?film  instance of   movie
    ?film    wdt:P31       wd:Q11424;
    #        composer      Hans Zimmer
             wdt:P86       wd:Q76364;
             wdt:P86       ?composer.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    OPTIONAL {
        ?composer wdt:P569     ?dob.
    }
    OPTIONAL {
        ?composer wdt:P570     ?dod.
    }
}
GROUP BY ?composer ?composerLabel
HAVING (?composer != wd:Q76364)
ORDER BY ASC(?title)

,composerLabel,dob,dod
0,Pharrell Williams,1973-04-05T00:00:00Z,None
1,Junkie XL,1967-12-08T00:00:00Z,None
2,Thomas Newman,1955-10-20T00:00:00Z,None
3,Harold Faltermeyer,1952-10-05T00:00:00Z,None
4,Stanley Myers,1930-10-06T00:00:00Z,1993-11-09T00:00:00Z
5,Benjamin Wallfisch,1979-08-07T00:00:00Z,None
6,Lisa Gerrard,1961-04-12T00:00:00Z,None
7,Klaus Badelt,1967-06-12T00:00:00Z,None
8,James Newton Howard,1951-06-09T00:00:00Z,None
9,Alfredo Capalani,1854-06-19T00:00:00Z,1893-08-07T00:00:00Z


##CONSULTA 10
¿Cuántos asertos hay sobre Zimmer en Wikidata? Ten en cuenta que Zimmer puede
aparecer tanto como sujeto como objeto de cada tripleta.

In [ ]:
%%wdsparql
SELECT (COUNT(*) AS ?nasserts)
WHERE
{   
    BIND(wd:Q76364 AS ?hanszimmer)
    ?hanszimmer ?assert1 ?item.
    ?subject    ?assert2 ?hanszimmer.
}

,nasserts
0,346882
